In [ ]:
#This example notebook was downloaded from the following link:
# https://colab.research.google.com/drive/1EMgPuFaD-xpboG_ZwZcytnlOlr39rakd 

# Hands on with Graph Neural Networks

# **Installing Pytorch Geometric**

# Enforce pytorch version 1.6.0
import torch
if torch.__version__ != '1.6.0':
  !pip uninstall torch -y
  !pip uninstall torchvision -y
  !pip install torch==1.6.0
  !pip install torchvision==0.7.0

# Check pytorch version and make sure you use a GPU Kernel
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"
!python --version
!nvidia-smi

In [1]:
import torch

C:\Users\amc121\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Make sure you clicked "RESTART RUNTIME" above (if torch version was different)!

# If something breaks in the notebook it is probably related to a mismatch between the Python version, CUDA or torch
import torch
pytorch_version = f"torch-{torch.__version__}+cu{torch.version.cuda.replace('.', '')}.html"
!pip install --no-index torch-scatter -f https://pytorch-geometric.com/whl/$pytorch_version
!pip install --no-index torch-sparse -f https://pytorch-geometric.com/whl/$pytorch_version
!pip install --no-index torch-cluster -f https://pytorch-geometric.com/whl/$pytorch_version
!pip install --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/$pytorch_version
!pip install torch-geometric


## Looking into the Dataset

In [2]:
from torch_geometric.datasets import MNISTSuperpixels
from torch_geometric.data import DataLoader
 
# Load the MNISTSuperpixel dataset
data = MNISTSuperpixels(root=".")
data

MNISTSuperpixels(60000)

In [3]:
# Investigating the dataset
print("Dataset type: ", type(data))
print("Dataset features: ", data.num_features)
print("Dataset target: ", data.num_classes)
print("Dataset length: ", data.len)
print("Dataset sample: ", data[0])
print("Sample  nodes: ", data[0].num_nodes)
print("Sample  edges: ", data[0].num_edges)

# edge_index = graph connections
# x = node features (75 nodes have each 1 feature)
# y = labels (dimension)

Dataset type:  <class 'torch_geometric.datasets.mnist_superpixels.MNISTSuperpixels'>
Dataset features:  1
Dataset target:  10
Dataset length:  <bound method InMemoryDataset.len of MNISTSuperpixels(60000)>
Dataset sample:  Data(x=[75, 1], edge_index=[2, 1399], y=[1], pos=[75, 2])
Sample  nodes:  75
Sample  edges:  1399


In [4]:
data[1]

Data(x=[75, 1], edge_index=[2, 1260], y=[1], pos=[75, 2])

In [5]:
# Investigating the features
# Shape: [num_nodes, num_node_features]
data[0].x

tensor([[0.6005],
        [0.7350],
        [0.5124],
        [0.7634],
        [0.5810],
        [0.7941],
        [0.7025],
        [0.6235],
        [0.7882],
        [0.5838],
        [0.5105],
        [0.2810],
        [0.5782],
        [0.7797],
        [0.4412],
        [0.6409],
        [0.2843],
        [0.7843],
        [0.6980],
        [0.5314],
        [0.8451],
        [0.7182],
        [0.2941],
        [0.6033],
        [0.8843],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0

In [6]:

# Shape [2, num_edges]
data[0].edge_index.t()

tensor([[ 0,  3],
        [ 0,  8],
        [ 0, 10],
        ...,
        [74, 55],
        [74, 63],
        [74, 69]])

In [7]:
data[0].y

tensor([5])

# Implementing Graph Neural Network
Building a Graph Neural Network works the same way as building a Convolutional Neural Network, we simple add some layers.

The GCN simply extends torch.nn.Module. GCNConv expects:

    in_channels = Size of each input sample.
    out_channels = Size of each output sample.

We apply three convolutional layers, which means we learn the information about 3 neighbor hops. After that we apply a pooling layer to combine the information of the individual nodes, as we want to perform graph-level prediction.

Always keep in mind that different learning problems (node, edge or graph prediction) require different GNN architectures.

For example for node-level prediction you will often encounter masks. For graph-level predictions on the other hand you need to combine the node embeddings.


In [8]:
import torch
from torch.nn import Linear
import torch.nn.functional as F 
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
embedding_size = 64
class GCN(torch.nn.Module):
    def __init__(self):
        # Init parent
        super(GCN, self).__init__()
        torch.manual_seed(42)
        # GCN layers
        self.initial_conv = GCNConv(data.num_features, embedding_size)
        self.conv1 = GCNConv(embedding_size, embedding_size)
        self.conv2 = GCNConv(embedding_size, embedding_size)
        self.conv3 = GCNConv(embedding_size, embedding_size)
        # Output layer
        self.out = Linear(embedding_size*2, data.num_classes)
    def forward(self, x, edge_index, batch_index):
        # First Conv layer
        
        print('x.shape_inicial: ', x.shape) #ERASE THIS
        
        hidden = self.initial_conv(x, edge_index)
        hidden = F.tanh(hidden)
        
        print('x.shape_pre_conv1: ', x.shape) #ERASE THIS
        
        # Other Conv layers
        hidden = self.conv1(hidden, edge_index)
        hidden = F.tanh(hidden)
        
        print('x.shape_conv1: ', x.shape) #ERASE THIS
        
        hidden = self.conv2(hidden, edge_index)
        hidden = F.tanh(hidden)
        hidden = self.conv3(hidden, edge_index)
        hidden = F.tanh(hidden)
        
        print('x.shape_conv3: ', x.shape) #ERASE THIS
        
        # Global Pooling (stack different aggregations)
        hidden = torch.cat([gmp(hidden, batch_index), 
                            gap(hidden, batch_index)], dim=1)
        
        print('x.shape_pooling: ', x.shape) #ERASE THIS
        
        # Apply a final (linear) classifier.
        out = self.out(hidden)
        
        print('x.shape_final: ', x.shape) #ERASE THIS
        
        return out, hidden
model = GCN()
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GCN(
  (initial_conv): GCNConv(1, 64)
  (conv1): GCNConv(64, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (out): Linear(in_features=128, out_features=10, bias=True)
)
Number of parameters:  13898


In [9]:
data_size = len(data)

In [10]:
#pause_here

## Training the GNN

In [11]:
data=data[:int(data_size * 0.15)]

In [12]:
len(data)

9000

In [13]:
data

MNISTSuperpixels(9000)

In [14]:
data[100]

Data(x=[75, 1], edge_index=[2, 1372], y=[1], pos=[75, 2])

In [15]:
4800/75

64.0

In [16]:
from torch_geometric.data import DataLoader
import warnings
warnings.filterwarnings("ignore")

# Cross EntrophyLoss
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0007)  
# Use GPU for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# Wrap data in a data loader
data_size = len(data)
NUM_GRAPHS_PER_BATCH = 64
loader = DataLoader(data[:int(data_size * 0.8)], 
                    batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
test_loader = DataLoader(data[int(data_size * 0.8):], 
                         batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)

def train(data):
    # Enumerate over the data
    for batch in loader:
      # Use GPU
      batch.to(device)  
      # Reset gradients
      optimizer.zero_grad() 
      # Passing the node features and the connection info
      print('entries: ',batch.x.float().shape, batch.edge_index.shape, batch.batch.shape) #ERASE THIS
      pred, embedding = model(batch.x.float(), batch.edge_index, batch.batch) 
      # Calculating the loss and gradients
      loss = torch.sqrt(loss_fn(pred, batch.y))       
      loss.backward()  
      # Update using the gradients
      optimizer.step()   
    return loss, embedding
print("Starting training...")
losses = []
for epoch in range(5):
    loss, h = train(data)
    losses.append(loss)
    if epoch % 1 == 0:
      print(f"Epoch {epoch} | Train Loss {loss}")

Starting training...
entries:  torch.Size([4800, 1]) torch.Size([2, 88464]) torch.Size([4800])
x.shape_inicial:  torch.Size([4800, 1])
x.shape_pre_conv1:  torch.Size([4800, 1])
x.shape_conv1:  torch.Size([4800, 1])
x.shape_conv3:  torch.Size([4800, 1])
x.shape_pooling:  torch.Size([4800, 1])
x.shape_final:  torch.Size([4800, 1])
entries:  torch.Size([4800, 1]) torch.Size([2, 88885]) torch.Size([4800])
x.shape_inicial:  torch.Size([4800, 1])
x.shape_pre_conv1:  torch.Size([4800, 1])
x.shape_conv1:  torch.Size([4800, 1])
x.shape_conv3:  torch.Size([4800, 1])
x.shape_pooling:  torch.Size([4800, 1])
x.shape_final:  torch.Size([4800, 1])
entries:  torch.Size([4800, 1]) torch.Size([2, 90547]) torch.Size([4800])
x.shape_inicial:  torch.Size([4800, 1])
x.shape_pre_conv1:  torch.Size([4800, 1])
x.shape_conv1:  torch.Size([4800, 1])
x.shape_conv3:  torch.Size([4800, 1])
x.shape_pooling:  torch.Size([4800, 1])
x.shape_final:  torch.Size([4800, 1])
entries:  torch.Size([4800, 1]) torch.Size([2, 89

x.shape_conv3:  torch.Size([4800, 1])
x.shape_pooling:  torch.Size([4800, 1])
x.shape_final:  torch.Size([4800, 1])
entries:  torch.Size([4800, 1]) torch.Size([2, 89863]) torch.Size([4800])
x.shape_inicial:  torch.Size([4800, 1])
x.shape_pre_conv1:  torch.Size([4800, 1])
x.shape_conv1:  torch.Size([4800, 1])
x.shape_conv3:  torch.Size([4800, 1])
x.shape_pooling:  torch.Size([4800, 1])
x.shape_final:  torch.Size([4800, 1])
entries:  torch.Size([4800, 1]) torch.Size([2, 89469]) torch.Size([4800])
x.shape_inicial:  torch.Size([4800, 1])
x.shape_pre_conv1:  torch.Size([4800, 1])
x.shape_conv1:  torch.Size([4800, 1])
x.shape_conv3:  torch.Size([4800, 1])
x.shape_pooling:  torch.Size([4800, 1])
x.shape_final:  torch.Size([4800, 1])
entries:  torch.Size([4800, 1]) torch.Size([2, 89231]) torch.Size([4800])
x.shape_inicial:  torch.Size([4800, 1])
x.shape_pre_conv1:  torch.Size([4800, 1])
x.shape_conv1:  torch.Size([4800, 1])
x.shape_conv3:  torch.Size([4800, 1])
x.shape_pooling:  torch.Size([48

x.shape_conv3:  torch.Size([4800, 1])
x.shape_pooling:  torch.Size([4800, 1])
x.shape_final:  torch.Size([4800, 1])
entries:  torch.Size([4800, 1]) torch.Size([2, 89520]) torch.Size([4800])
x.shape_inicial:  torch.Size([4800, 1])
x.shape_pre_conv1:  torch.Size([4800, 1])
x.shape_conv1:  torch.Size([4800, 1])
x.shape_conv3:  torch.Size([4800, 1])
x.shape_pooling:  torch.Size([4800, 1])
x.shape_final:  torch.Size([4800, 1])
entries:  torch.Size([4800, 1]) torch.Size([2, 89308]) torch.Size([4800])
x.shape_inicial:  torch.Size([4800, 1])
x.shape_pre_conv1:  torch.Size([4800, 1])
x.shape_conv1:  torch.Size([4800, 1])
x.shape_conv3:  torch.Size([4800, 1])
x.shape_pooling:  torch.Size([4800, 1])
x.shape_final:  torch.Size([4800, 1])
entries:  torch.Size([4800, 1]) torch.Size([2, 89097]) torch.Size([4800])
x.shape_inicial:  torch.Size([4800, 1])
x.shape_pre_conv1:  torch.Size([4800, 1])
x.shape_conv1:  torch.Size([4800, 1])
x.shape_conv3:  torch.Size([4800, 1])
x.shape_pooling:  torch.Size([48

x.shape_conv3:  torch.Size([4800, 1])
x.shape_pooling:  torch.Size([4800, 1])
x.shape_final:  torch.Size([4800, 1])
entries:  torch.Size([4800, 1]) torch.Size([2, 87742]) torch.Size([4800])
x.shape_inicial:  torch.Size([4800, 1])
x.shape_pre_conv1:  torch.Size([4800, 1])
x.shape_conv1:  torch.Size([4800, 1])
x.shape_conv3:  torch.Size([4800, 1])
x.shape_pooling:  torch.Size([4800, 1])
x.shape_final:  torch.Size([4800, 1])
entries:  torch.Size([4800, 1]) torch.Size([2, 89636]) torch.Size([4800])
x.shape_inicial:  torch.Size([4800, 1])
x.shape_pre_conv1:  torch.Size([4800, 1])
x.shape_conv1:  torch.Size([4800, 1])
x.shape_conv3:  torch.Size([4800, 1])
x.shape_pooling:  torch.Size([4800, 1])
x.shape_final:  torch.Size([4800, 1])
entries:  torch.Size([4800, 1]) torch.Size([2, 88843]) torch.Size([4800])
x.shape_inicial:  torch.Size([4800, 1])
x.shape_pre_conv1:  torch.Size([4800, 1])
x.shape_conv1:  torch.Size([4800, 1])
x.shape_conv3:  torch.Size([4800, 1])
x.shape_pooling:  torch.Size([48


KeyboardInterrupt



In [18]:
for batch in loader:
    print(batch.size())

(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)
(4800, 4800)

In [ ]:
NUM_GRAPHS_PER_BATCH = 64
data_temp = data[:int(data_size * 0.8)]
loader = DataLoader(data_temp, 
                    batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)

In [ ]:
data_temp[0].x.t()

In [ ]:
for batch in loader:
    batch.x
    break

In [ ]:
batch

In [ ]:
batch.x.t().shape

### Visualizing the Training loss

In [ ]:
# Visualize learning (training loss)
import seaborn as sns
losses_float = [float(loss.cpu().detach().numpy()) for loss in losses] 
loss_indices = [i for i,l in enumerate(losses_float)] 
plt = sns.lineplot(loss_indices, losses_float)
plt



In [ ]:
pause_here

### Getting a test prediction

In [ ]:
import pandas as pd 
test_batch = next(iter(test_loader))
with torch.no_grad():
    test_batch.to(device)
    pred, embed = model(test_batch.x.float(), test_batch.edge_index, test_batch.batch) 
    pred=torch.argmax(pred,dim=1)
print(test_batch.y[0])#Actual REsult
print(pred[0])#Predicted Result


In [ ]:
import torch
import networkx as nx
import matplotlib.pyplot as plt


def visualize(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])

    if torch.is_tensor(h):
        h = h.detach().cpu().numpy()
        plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
        if epoch is not None and loss is not None:
            plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    else:
        nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                         node_color=color, cmap="Set2")
    plt.show()

In [ ]:
data

In [ ]:
dataset=data[1]
print(f'Is undirected: {dataset.is_undirected()}')
from torch_geometric.utils import to_networkx
G = to_networkx(dataset, to_undirected=True)
 
visualize(G, "yellow") 
visualize(G, "red")

In [ ]:
dataset.y